In [1]:
import sys, os
sys.path.append(os.getcwd())
sys.path.append('../..')
from imports import ilpgraph, readFile
from network import Steiner_Linear, Steiner_Linear_tightened
import networkx as nx
from gurobipy import *
import xlsxwriter
import datetime

oldtime = None
oldsol = 10000000000

# Terminates the model calculation if no better solution than the current optimal could be found within 60 seconds.
def terminateCheck(model, where):
    global oldtime
    global oldsol
    timenow = datetime.datetime.now()

    if where == GRB.Callback.MIP:
        newsol = model.cbGet(GRB.Callback.MIP_OBJBST)
        diff = ((timenow - oldtime).total_seconds()) / 60.0
    
        # If more than 30 seconds have passed since the past solution
        if oldsol == newsol and diff > 1:
            print(oldtime)
            print(timenow)
            print(diff)
            model.terminate()

    elif where == GRB.Callback.MIPSOL:
        oldtime = timenow
        oldsol = model.cbGet(GRB.Callback.MIPSOL_OBJBST)
        print("new Sol ", oldsol)
        print("time ", oldtime)

def __write_solution(solution:list):
    
    workbook = xlsxwriter.Workbook('Benchmarks_SteinFlow.xlsx', {'nan_inf_to_errors': True})
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('D1', 'LinearModel', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('D1', 'Tightened', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    worksheet.write('E1', 'Runtime', bold)

    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    workbook.close()
    print("Finished!")
    
    
def runBenchmark(paths:str, linesToRead:range):
    global oldtime
    solutions = []
    lines = []
    for path in paths:
        for i in range(len(linesToRead)):
            G, terminals = readFile.stp_to_networkx(path)
            Graph = ilpgraph.ILPGraph()
            Graph.setNXGraph(G)
            Graph.setEdgeVars(G.edges())
            Graph.setNodeVars(G.nodes())
            
            modelLin = Steiner_Linear.createModel(Graph, terminals)
            modelLin.Params.threads = 3
            modelLin.params.TimeLimit = 300
            oldtime = datetime.datetime.now()
            modelLin.optimize(terminateCheck) 
            
            modelTight = Steiner_Linear_tightened.createModel(Graph, terminals)
            modelTight.Params.threads = 3
            modelTight.params.TimeLimit = 300
            oldtime = datetime.datetime.now()
            modelTight.optimize(terminateCheck) 
                
                
            solutions.append((i, len(G.edges()), len(G.nodes()), \
                modelLin.objVal, modelLin.numConstrs, modelLin.MIPGap, modelLin.runTime, \
                modelTight.objVal, modelTight.numConstrs, modelTight.MIPGap, modelTight.runTime))
        print(solutions)
    __write_solution(solutions)  
    

if __name__ == '__main__':
    linesToRead = range(0,1)
    
    # Input Files Path
    paths = [r'../Steiner_testInstances/d09.stp',
            r'../Steiner_testInstances/d13.stp',
            r'../Steiner_testInstances/e08.stp',
            r'../Steiner_testInstances/e10.stp',
            r'../Steiner_testInstances/e13.stp',
            r'../Steiner_testInstances/e15.stp',
            r'../Steiner_testInstances/e16.stp',
            r'../Steiner_testInstances/e17.stp',
            r'../Steiner_testInstances/e18.stp',
            r'../Steiner_testInstances/e19.stp',
            r'../Steiner_testInstances/e20.stp',]

    runBenchmark(paths, linesToRead)
    




Using license file /opt/gurobi/gurobi.lic
Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 18251 rows, 6000 columns and 70250 nonzeros
Model fingerprint: 0x1a3429e3
Variable types: 0 continuous, 6000 integer (5000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 9354 rows and 330 columns
Presolve time: 0.16s
Presolved: 8897 rows, 5670 columns, 38531 nonzeros
Variable types: 0 continuous, 5670 integer (4671 binary)

Root relaxation: objective 1.436000e+03, 6556 iterations, 0.32 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Ob

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  497.00000    0   60          -  497.00000      -     -    1s
     0     0  497.00000    0  113          -  497.00000      -     -    1s
     0     0  497.00000    0  113          -  497.00000      -     -    1s
     0     0  497.00000    0   76          -  497.00000      -     -    2s
     0     0  497.00000    0  106          -  497.00000      -     -    2s
     0     0  497.00000    0   25          -  497.00000      -     -    2s
     0     0  497.00000    0   23          -  497.00000      -     -    2s
     0     0  497.00000    0   83          -  497.00000      -     -    2s
     0     0  497.00000    0   71          -  497.00000      -     -    2s
     0     0  497.00000    0   75          -  497.00000      -     -    2s
     0     0  497.00000    0   76          -  497.00000      -     -    3s
     0     0  497.00000    0   64          -  497.00000      -     -    3s
     0     0  497.0000

  RHS range        [1e+00, 2e+03]
Presolve removed 22606 rows and 652 columns
Presolve time: 0.42s
Presolved: 22812 rows, 14348 columns, 98802 nonzeros
Variable types: 0 continuous, 14348 integer (11852 binary)

Root relaxation: objective 2.631000e+03, 13962 iterations, 1.82 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2631.00000    0  156          - 2631.00000      -     -    2s
     0     0 2631.00000    0  169          - 2631.00000      -     -    2s
     0     0 2631.00000    0  137          - 2631.00000      -     -    3s
     0     0 2631.00000    0  134          - 2631.00000      -     -    3s
     0     0 2631.00000    0   68          - 2631.00000      -     -    3s
     0     0 2631.00000    0   68          - 2631.00000      -     -    4s
     0     0 2631.00000    0   54          - 2631.00000      -     -    4s
     0     0 2631.00000    0   57      

Presolve time: 0.39s
Presolved: 20508 rows, 13493 columns, 87809 nonzeros
Variable types: 0 continuous, 13493 integer (10999 binary)

Root relaxation: objective 5.593000e+03, 14737 iterations, 1.17 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5593.00000    0  154          - 5593.00000      -     -    2s
     0     0 5593.00000    0  178          - 5593.00000      -     -    2s
     0     0 5593.00000    0  218          - 5593.00000      -     -    2s
     0     0 5593.00000    0  292          - 5593.00000      -     -    2s
     0     0 5593.00000    0  112          - 5593.00000      -     -    3s
     0     0 5593.00000    0  130          - 5593.00000      -     -    3s
     0     0 5593.00000    0  366          - 5593.00000      -     -    3s
     0     0 5593.00000    0  151          - 5593.00000      -     -    3s
     0     0 5593.00000    0  151         

Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 105418 rows, 30000 columns and 430417 nonzeros
Model fingerprint: 0x9f1de9f7
Variable types: 0 continuous, 30000 integer (27500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 2e+03]
Presolve removed 54243 rows and 417 columns
Presolve time: 0.96s
Presolved: 51175 rows, 29583 columns, 244400 nonzeros
Variable types: 0 continuous, 29583 integer (27083 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.06s

Solved with primal simplex

Root relaxation: objective 1.277000e+03, 25695 

...

     0     0   24.33333    0  461   29.00000   24.33333  16.1%     -  151s
     0     0   24.33333    0   43   29.00000   24.33333  16.1%     -  156s
new Sol  25.0
time  2021-03-23 10:14:51.987522
H    0     0                      25.0000000   24.33333  2.67%     -  156s
     0     0   24.33333    0   43   25.00000   24.33333  2.67%     -  156s

Cutting planes:
  Implied bound: 89
  Zero half: 12
  RLT: 5

Explored 1 nodes (140262 simplex iterations) in 156.96 seconds
Thread count was 3 (of 12 available processors)

Solution count 3: 25 29 33 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.500000000000e+01, best bound 2.500000000000e+01, gap 0.0000%

User-callback calls 29375, time in user-callback 0.55 sec
[(0, 2000, 1000, 1448.0, 18251, 0.006906077348066298, 62.94248008728027, 1448.0, 26251, 0.006906077348066298, 67.78062009811401), (0, 5000, 1000, 500.0, 42168, 0.005999999999998977, 70.17797994613647, 500.0, 62168, 0.005999999999999431, 82.39623212814331), (0, 5000, 2

  RHS range        [1e+00, 5e+03]
Presolve removed 351299 rows and 417 columns (presolve time = 5s) ...
Presolve removed 394608 rows and 417 columns (presolve time = 11s) ...
Presolve removed 394608 rows and 417 columns
Presolve time: 10.96s
Presolved: 360810 rows, 129583 columns, 1572004 nonzeros
Variable types: 0 continuous, 129583 integer (127083 binary)

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                         19s
    2471    1.6956700e+32   3.614391e+33   1.695670e+02     20s
   22438    7.9783040e+02   1.068081e+03   0.000000e+00     25s
   29539    5.9502436e+02   3.854523e+03   0.000000e+00     30s
   33196    5.7059817e+02   7.544401e+02   0.000000e+00     36s
   36936    5.6239446e+02   2.131011e+03   0.000000e+00     40s
   39743    5.6123650e+02   0.000000e+00   0.000000e+00

     0     0  757.00000    0   79  758.00000  757.00000  0.13%     -   51s
     0     0  757.00000    0   78  758.00000  757.00000  0.13%     -   51s
     0     2  757.00000    0   78  758.00000  757.00000  0.13%     -   51s
  1011   771  757.00000  148   19  758.00000  757.00000  0.13%  23.9   55s
  2694  1639  757.00000  210    8  758.00000  757.00000  0.13%  39.1   60s
  7403  2811 infeasible  211       758.00000  757.00000  0.13%  20.2   65s
 11775  3521 infeasible  200       758.00000  757.00000  0.13%  17.3   70s
 16998  4297  757.00000  203   18  758.00000  757.00000  0.13%  15.3   75s
 17795  4298  757.00000  198  197  758.00000  757.00000  0.13%  14.9   86s
 17808  4307  757.00000  102   81  758.00000  757.00000  0.13%  14.9   90s
 18036  4701  757.00000   68  127  758.00000  757.00000  0.13%  16.0   95s
2021-03-23 10:38:12.883728
2021-03-23 10:39:13.242543
1.00598025
 21951  6691  757.00000  161   24  758.00000  757.00000  0.13%  15.9  100s

Cutting planes:
  Gomory: 2
  Impl

Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 506251 rows, 130000 columns and 2131250 nonzeros
Model fingerprint: 0xce3d14b1
Variable types: 0 continuous, 130000 integer (127500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 2e+03]
Presolve removed 298171 rows and 1250 columns (presolve time = 5s) ...
Presolve removed 298171 rows and 1250 columns
Presolve time: 7.50s
Presolved: 208080 rows, 128750 columns, 1097353 nonzeros
Variable types: 0 continuous, 128750 integer (126250 binary)

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing first log only...

     0     0 1341.00000    0  459 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0  129 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0  373 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0  674 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0  154 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0  669 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0  103 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0  999 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0   43 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0   29 1342.00000 1341.00000  0.07%     -  177s
     0     0 1341.00000    0   19 1342.00000 1341.00000  0.07%     -  178s
     0     0 1341.00000    0  142 1342.00000 1341.00000  0.07%     -  178s
     0     0 1341.00000    0   16 1342.00000 1341.00000  0.07%     -  178s
     0     0 1341.00000  